In [2]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torchvision import datasets
import os
import numpy as np

In [3]:
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
num_classes = 10
model.fc = nn.Linear(512, num_classes)

In [6]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
for params in model.parameters():
  params.requires_grad_(False)

In [8]:
for params in model.fc.parameters():
  params.requires_grad_(True)

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters())

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/AIP/Train_Test_dataset.zip'

In [24]:
data_dir = '/content/classification_dataset'
train_dir = "/content/classification_dataset/train"
test_dir = "/content/classification_dataset/test"

In [28]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]), 'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train','test']} ##### os.path.join( path, images )


In [ ]:
image_datasets

In [43]:
train_dataloader = torch.utils.data.DataLoader(image_datasets['train'], batch_size = 64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(image_datasets["test"], batch_size = 64, shuffle=True)

In [ ]:
for input, labels in test_dataloader:
  print(labels)

In [13]:
if torch.cuda.is_available(): 
  device = 'cuda'
else:
  device = 'cpu'

In [49]:
epochs = 100
for epoch in range(epochs):
  for inputs, labels in train_dataloader:
    model.train()

    optimizer.zero_grad()

    y_pred = model(inputs.to(device))

    loss = loss_fn(y_pred, labels.to(device))

    loss.backward()

    optimizer.step()
  
  model.eval()
  with torch.inference_mode():
    correct = 0
    total = 0
    for inputs, labels in test_dataloader:
      y_pred = model(inputs.to(device))
      _, predicted = torch.max(y_pred.data, 1)
      total += labels.size(0)
      correct += (predicted == labels.to(device)).sum().item()
    print('Test accuracy for {epoch} epoch: {:.2f}%'.format(100 * correct / total))

Test accuracy: 99.17%
Test accuracy: 97.50%
Test accuracy: 99.17%
Test accuracy: 99.17%
Test accuracy: 98.33%
Test accuracy: 99.17%
Test accuracy: 100.00%
Test accuracy: 99.17%
Test accuracy: 97.50%
Test accuracy: 98.33%
Test accuracy: 98.33%
Test accuracy: 98.33%
Test accuracy: 100.00%
Test accuracy: 98.33%
Test accuracy: 97.50%
Test accuracy: 100.00%
Test accuracy: 98.33%
Test accuracy: 98.33%
Test accuracy: 97.50%
Test accuracy: 100.00%
Test accuracy: 98.33%
Test accuracy: 99.17%
Test accuracy: 99.17%
Test accuracy: 99.17%
Test accuracy: 100.00%
Test accuracy: 97.50%
Test accuracy: 100.00%
Test accuracy: 96.67%
Test accuracy: 99.17%
Test accuracy: 99.17%
Test accuracy: 98.33%
Test accuracy: 99.17%
Test accuracy: 97.50%
Test accuracy: 100.00%
Test accuracy: 100.00%
Test accuracy: 100.00%
Test accuracy: 99.17%
Test accuracy: 99.17%
Test accuracy: 98.33%
Test accuracy: 100.00%
Test accuracy: 98.33%
Test accuracy: 100.00%
Test accuracy: 100.00%
Test accuracy: 100.00%
Test accuracy: 100.